- 1. Update config.yaml 

- 2. Update entity

In [1]:
import os

In [2]:
os.getcwd()

'c:\\TheBritishCollege\\DataScience\\Kidney_disease_classification\\research'

In [3]:
os.chdir('../')

In [4]:
os.getcwd()

'c:\\TheBritishCollege\\DataScience\\Kidney_disease_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelTrainingConfig: 
    root_dir: Path
    trained_model_path: Path
    params_augmentation: bool
    params_image_size: list
    params_epochs: int
    params_batch_size: int
    training_data: Path
    final_model_train: Path



- 3. Update configuration Manger

In [7]:
from src.cnn_classifier.constants import *
from src.cnn_classifier.utils.common import read_yaml, create_directories
from src.cnn_classifier import logger

In [25]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_training_config(self)-> ModelTrainingConfig:
        
        # config_train=self.config.training
        params=self.params
        prepare_base_model=self.config.prepare_base_model
        training_data_path=os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([self.config.training.root_dir])
        try: 
            model_training_config=ModelTrainingConfig(
                root_dir=Path(self.config.training.root_dir),
                trained_model_path=Path(prepare_base_model.updated_base_model_path),
                training_data=Path(training_data_path),
                final_model_train=Path(self.config.training.trained_model_path),
                
                
                params_augmentation= params.AUGMENTATION,
                params_batch_size=params.BATCH_SIZE,
                params_image_size=params.IMAGE_SIZE,
                params_epochs=params.EPOCHS
                )
            return model_training_config
        except Exception as e: 
            raise e
            
            

- 4. Update Components

In [26]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [33]:
class Training: 
    def __init__(self, config: ModelTrainingConfig): 
        self.config=config
    
    def get_base_model(self): 
        self.model=tf.keras.models.load_model(
            self.config.trained_model_path
            )
    
    def train_valid_generator(self): 

        datagenerator_kwargs= dict(
            rescale=1./255,
            validation_split=0.20
            )
        logger.info(f"Train_valid generator : {datagenerator_kwargs}")
        
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        logger.info(f"Data flow kwargs : {dataflow_kwargs}")

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    
    def train(self):
            self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
            self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

            logger.info(f"In train function steps_per_epoch: {self.steps_per_epoch}")
            logger.info(f"In train function steps_per_epoch: {self.validation_steps}")



            self.model.fit(
                self.train_generator,
                epochs=self.config.params_epochs,
                steps_per_epoch=self.steps_per_epoch,
                validation_steps=self.validation_steps,
                validation_data=self.valid_generator
            )

            self.save_model(
                path=self.config.final_model_train
                model=self.model
            )

In [34]:
try:
    config = ConfigurationManager()
    training_config = config.get_model_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2023-10-12 08:38:22,866 - INFO, - common - yaml file : config\config.yaml loaded sucessfully]
[2023-10-12 08:38:22,871 - INFO, - common - yaml file : params.yaml loaded sucessfully]
[2023-10-12 08:38:22,877 - INFO, - common - Created directories at : artifacts]
[2023-10-12 08:38:22,879 - INFO, - common - Created directories at : artifacts/training]


[2023-10-12 08:38:24,461 - INFO, - 684293453 - Train_valid generator : {'rescale': 0.00392156862745098, 'validation_split': 0.2}]
[2023-10-12 08:38:24,463 - INFO, - 684293453 - Data flow kwargs : {'target_size': [224, 224], 'batch_size': 16, 'interpolation': 'bilinear'}]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
[2023-10-12 08:38:24,551 - INFO, - 684293453 - In train function steps_per_epoch: 23]
[2023-10-12 08:38:24,554 - INFO, - 684293453 - In train function steps_per_epoch: 5]
23/23 [==============================] - 809s 36s/step - loss: 10.0724 - accuracy: 0.6096 - val_loss: 17.6537 - val_accuracy: 0.4750
